In [6]:
import os
import pandas as pd
import numpy as np
import re
import dask.dataframe as dd

# path = 'C:/Users/user/Desktop/AIML/datasets/Data_Preprocessing/spam_set'
path = '/home/maroco/dataset/spam_dataset'
files = os.listdir(path)
filenames = []

for root, dirs, files in os.walk(path):
  for file in files:
    if file.endswith('.csv'):
      filenames.append(os.path.join(root, file))

# Create a list to store DataFrames
all_dfs = []

for file in filenames:
  try:
    # Use 'on_bad_lines' instead of 'error_bad_lines'
    # df = pd.read_csv(file, on_bad_lines='skip', delimiter=",")  # Skip bad lines
    df = pd.read_csv(file)

    # 정상화
    df.columns=['index', 'data']
    df = df.set_index('index')
    df['spam'] = 1
    df['data'] = df['data'].str.replace('ifg@','')

    # Dask Dataframe 생성
    # Move ddf creation after modifying the Pandas DataFrame
    ddf = dd.from_pandas(df, npartitions=4) # Create ddf after modifications


    # Append the current DataFrame to the list
    result = ddf.compute()
    all_dfs.append(result)

  except pd.errors.ParserError as e:
    print(f"Error reading file {file}: {e}")

# Concatenate all DataFrames in the list into a single DataFrame
raw = pd.concat(all_dfs, ignore_index=True)

In [7]:
# 텍스트 처리 함수 설정
def preprocess_text(text):
    # 여러 개의 공백을 하나의 공백으로 축소
    text = re.sub(r'\s+', ' ', text)
    # 앞뒤 공백 제거
    text = text.strip()
    # 맨 앞의 숫자와 공백 또는 콜론 제거
    text = re.sub(r'^\d+\s*', '', text)
    # 필요에 따라 특수 문자 제거 (예시로 일부 문자만 제거)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', text)
    return text

In [8]:
# 결측치 처리
print(len(raw))
drop_index = raw[raw['data'].isnull()].index
raw = raw.drop(drop_index)
print(len(raw))

159507
159507


In [9]:
# 중복 내용 제거
print(len(raw))
raw = raw.drop_duplicates('data')
print(len(raw))

159507
158337


In [10]:
raw['data']=raw['data'].apply(preprocess_text)

In [13]:
raw['data'] = raw['data'].str.replace('Web발신', '')
raw['data'] = raw['data'].str.replace('국제발신', '')
raw['data'] = raw['data'].str.replace('광고', '')

In [14]:
raw.head()

,data,spam
0,Hil튼 scom열소l 캍옥 CDDC,1
1,정선생의 정보 제공 커뮤니티안녕하십니까정진홍 주식전문가입니다미국에서 인플레이션을 잡...,1
2,비번 bitlywBUinvip 주식 세력 단타 일매도체험방 입니다일만,1
3,원 으로원금보장지원금 원 마감문의하기 xxpeaskcom,1
4,맥심SLOTmexcom신규 만쿱폰매일 퍠이백,1


In [17]:
print(len(raw))

158337


In [16]:
raw.to_csv('/home/maroco/dataset/spam/spam_data.csv', index=False)